In [1]:
import keypress_recognition.models as models
from keypress_recognition import dataset_p as dataset
from keypress_recognition import dataset_t
import numpy as np
import matplotlib.pyplot as plt
import cv2
import torch
%load_ext autoreload
%autoreload 2

In [2]:
concat = False
delta = 3 if concat else 1
channels = 3 * delta

In [3]:
mod = {}
mod['black'] = models.get_black_bundle_model(channels)
mod['black'].load_model('keypress_recognition/models/best_black_bundle.tar')
mod['white'] = models.get_white_bundle_model(channels)
mod['white'].load_model('keypress_recognition/models/best_white_bundle.tar') 

Trying to load model from keypress_recognition/models/best_black_bundle.tar ...
done
Trying to load model from keypress_recognition/models/best_white_bundle.tar ...
done


In [11]:
dataset.load_all_data(spliter=['test', 'val'], color=['black', 'white'], size=['bundle'], keypress=True, concatenate=concat, delta=delta)

Start seperating keyboard ...
  White single width: 21px
  Black single width: 20px
  White bundle width: 37px
  Black bundle width: 36px


IntProgress(value=0, max=5000)

In testset: 
  # of pressed black key: 761
  # of unpressed black key: 179239
  # of pressed white key: 10831
  # of unpressed white key: 249169


IntProgress(value=0, max=2993)

In valset: 
  # of pressed black key: 817
  # of unpressed black key: 106931
  # of pressed white key: 5090
  # of unpressed white key: 150546


In [ ]:
result = np.zeros(dataset.X['bundle']['black']['test'])
for inputs, labels, ind in dataset.data_batch(type='test', size='bundle', color='black', need_velocity=False):
    _y = mod['black'].evaluate(inputs)
    result[ind] = _y
target = dataset.y['black']['test']
acc = np.empty((2, 2), dtype=int)
for i in (0, 1):
    for j in (0, 1):
        acc[i, j] = torch.sum((result == i) & (target == j)).tolist()
print(mod['black'].evaluate_accuracy_matrix(acc))

In [5]:
def visualize_keys(note):
    """
    note: bool
    return: str
    """
    return ("Y" if note else "N")

def display_concat_img(img, idx=None):
    if concat:
        if idx is None:
            return img[:, :, :, 3:6]
        else:
            return img[idx, :, :, 3:6]
    if idx is None:
        return img
    else:
        return img[idx, :, :, :]


In [66]:
for color in ('black', 'white'):
    print(color)
    results = mod[color].test(dataset, size='bundle', color=color, concatenate=concat)
    for k, v in results.items():
        print("{:10}: {:.4f}".format(k.title(), v))

black


IntProgress(value=0, max=358478)

[[178094    942]
 [  1170 178322]]
Total data count 358528 Batch size 64
Precision : 0.9935
Recall    : 0.9947
Accuracy  : 0.9941
F1 Score  : 0.9941
white


IntProgress(value=0, max=498338)

[[240311   7407]
 [  8873 241777]]
Total data count 498368 Batch size 64
Precision : 0.9646
Recall    : 0.9703
Accuracy  : 0.9673
F1 Score  : 0.9674


In [13]:
valresult = {}
for color in ('black', 'white'):
    dbatch = dataset.data_batch(type='val', size='bundle', color=color, need_velocity=False)
    valresult[color] = np.zeros(len(dataset.X['bundle'][color]['val']))
    for inputs, labels, ind in dbatch:
        _y = mod[color].evaluate(inputs, threshold=0.95 if color=='black' else 0.92).cpu().numpy() # (batchsize, )
        valresult[color][ind] = _y 
    target = dataset.y[color]['val'] > 0
    acc = np.empty((2, 2), dtype=int)
    for i in (0, 1):
        for j in (0, 1):
            acc[i, j] = np.sum(np.logical_and(valresult[color] == i, target == j))
    print(color)
    print(acc)
    print(mod[color].evaluate_accuracy_matrix(acc))

IntProgress(value=0, max=107748)

black
[[106798    139]
 [   133    678]]
(0.8360049321824907, 0.8298653610771114, 0.9974755911942681)


IntProgress(value=0, max=155636)

white
[[149350   1073]
 [  1196   4017]]
(0.770573566084788, 0.7891944990176817, 0.9854211107969878)


In [16]:
print(valresult['black'].shape)
print(valresult['black'].shape[0] / 36)
print(valresult['white'].shape[0] / 52)
reshaped = {
    'black': valresult['black'].reshape(2993, 36),
    'white': valresult['white'].reshape(2993, 52)
}
print(reshaped['white'].shape)

(107748,)
2993.0
2993.0
(2993, 52)


In [17]:
fuck = dataset_t.merge_two_colors(reshaped['white'], reshaped['black'])
assert fuck.shape == (2993, 88)
fucked = np.pad(fuck, ((0, 0), (21, 19)), mode='constant', constant_values=0)
print(fucked.shape)
np.save('./out/val_result_bundle.npy', fucked)
np.save('./out/val_result_black_bundle.npy', reshaped['black'])
np.save('./out/val_result_white_bundle.npy', reshaped['white'])

(2993, 128)


In [85]:
fuckyouresult = {}
for color in ('black', 'white'):
    dbatch = dataset.data_batch(type='test', size='bundle', color=color, need_velocity=False)
    fuckyouresult[color] = np.zeros(len(dataset.X['bundle'][color]['test']))
    for inputs, labels, ind in dbatch:
        _y = mod[color].evaluate(inputs, threshold=0.94 if color=='black' else 0.88).cpu().numpy() # (batchsize, )
        fuckyouresult[color][ind] = _y 
    target = dataset.y[color]['test'] > 0
    acc = np.empty((2, 2), dtype=int)
    for i in (0, 1):
        for j in (0, 1):
            acc[i, j] = np.sum(np.logical_and(fuckyouresult[color] == i, target == j))
    print(color)
    print(acc)
    print(mod[color].evaluate_accuracy_matrix(acc))

IntProgress(value=0, max=358478)

black
[[179136     90]
 [   103    671]]
(0.8669250645994832, 0.8817345597897503, 0.9989277777777777)


IntProgress(value=0, max=498338)

white
[[246805   2123]
 [  2364   8708]]
(0.7864884393063584, 0.8039885513802973, 0.9827423076923076)


In [87]:
print(fuckyouresult['black'].shape)
print(fuckyouresult['black'].shape[0] / 36)
print(fuckyouresult['white'].shape[0] / 52)
reshaped = {
    'black': fuckyouresult['black'].reshape(5000, 36),
    'white': fuckyouresult['white'].reshape(5000, 52)
}
print(reshaped['white'].shape)
print(reshaped['black'])

(180000,)
5000.0
5000.0
(5000, 52)
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [91]:
fuck = dataset_t.merge_two_colors(reshaped['white'], reshaped['black'])
assert fuck.shape == (5000, 88)
print(fuck.shape)
fucked = np.pad(fuck, ((0, 0), (21, 19)), mode='constant', constant_values=0)
print(fucked.shape)
np.save('./out/test_result_bundle.npy', fucked)
np.save('./out/test_result_black_bundle.npy', reshaped['black'])
np.save('./out/test_result_black_bundle.npy', reshaped['white'])

(5000, 88)
(5000, 128)


In [46]:
# ! we disable threshold at first

blacknpy = np.empty((0, 36))
whitenpy = np.empty((0, 52))
for whites, blacks in dataset_t.data_batch(size='bundle'):
    blackout = mod['black'].evaluate(blacks, threshold=None).cpu().numpy()
    whiteout = mod['white'].evaluate(whites, threshold=None).cpu().numpy()
#     out = dataset_t.merge_two_colors(whiteout, blackout)
    blacknpy = np.concatenate((blacknpy, blackout[None]), axis=0)
    whitenpy = np.concatenate((whitenpy, whiteout[None]), axis=0)
print(blacknpy.dtype, blacknpy.shape)

IntProgress(value=0, max=5000)

float64 (5000, 36)


In [ ]:
label = np.load('keypress_recognition/dataset/y_test/y_test.npy')
label = label[:, 21:109] > 0
print(label.shape)

black_mask = np.array([1, 4, 6, 9, 11, 13, 16, 18, 21, 23, 25, 28, 30, 33, 35, 37, 40, 42, 45, 47, 49, 52, 54, 57, 59, 61, 64,
    66, 69, 71, 73, 76, 78, 81, 83, 85])
white_mask = np.array([0, 2, 3, 5, 7, 8, 10, 12, 14, 15, 17, 19, 20, 22, 24, 26, 27, 29, 31, 32, 34, 36, 38, 39, 41, 43, 44, 46, 48,
    50, 51, 53, 55, 56, 58, 60, 62, 63, 65, 67, 68, 70, 72, 74, 75, 77, 79, 80, 82, 84, 86, 87])

In [40]:
blackpred = blacknpy > 0.5
print(dataset_t.evaluate(blackpred, label[:, black_mask]))
whitepred = whitenpy > 0.5
print(dataset_t.evaluate(whitepred, label[:, white_mask]))

{'accuracy': 0.9856944444444444, 'precision': 0.010787486515641856, 'recall': 0.026281208935611037, 'F1 score': 0.015296367112810709}
{'accuracy': 0.8991269230769231, 'precision': 0.10278637770897833, 'recall': 0.18391653586926415, 'F1 score': 0.1318724967726987}


In [30]:
pred = dataset_t.merge_two_colors(whitepred, blackpred)
print(pred.shape)
eva = dataset_t.evaluate(pred, label)
print(eva)

[[ True  True  True  True  True  True  True  True  True  True  True  True
   True  True  True  True  True  True  True  True  True  True  True  True
   True  True  True  True  True  True  True  True  True  True  True  True
   True  True  True  True  True  True  True  True  True  True  True  True
   True  True  True  True  True  True  True  True  True  True  True  True
   True  True  True  True  True  True  True  True  True  True  True  True
   True  True  True  True  True  True  True  True  True  True  True  True
   True  True  True  True]]
